<span><img align="left" src ="https://combilift.com/wp-content/uploads/2021/09/Combilift-Web-Logo_sm.png" width="170" /></span> <br/><br/>
### Data entry tool
<hr>

#### This tool is for adding tasks to the telematics project until a server is set up

### Login

In [13]:
# connect to api

%run ./DataAccess/CL_Login.ipynb

# connect to database

import mysql.connector

mydb = mysql.connector.connect(
    host="localhost",
    user="root",
    password="Z%Br6}5+cBg9yN<y",
    database="testdatabase"
)

print(mydb)


Connected machines: 170
Login: Success ✔


### Organization

In [14]:
%run ./DataAccess/CL_Get_Org.ipynb
%run ./DashboardWidgets/Widget_Org.ipynb

Select(description='Select Organisation', options=(('Meherrin River', '0ba082a0-df39-4021-b2df-be4ea9af6900'),…

### Model

In [15]:
%run ./DataAccess/CL_Get_Models.ipynb
%run ./DashboardWidgets/Widget_Model.ipynb

Select(description='Select Model', options=(('CB', '007d4401-82b6-47ab-8eb0-6552fa489764'), ('C Series', '0b08…

### Machine

In [16]:
%run ./DataAccess/CL_Get_Machine.ipynb
%run ./DashboardWidgets/Widget_Machine.ipynb

SelectMultiple(description='Select Machine', layout=Layout(height='initial'), options=(('MR4 4t 59265', '14755…

### Dates

In [17]:
%run ./DashboardWidgets/Widget_TimeFrame.ipynb
%run ./DashboardWidgets/timeseries_dropdown.ipynb

DatePicker(value=None, description='Start Date', step=1)

DatePicker(value=None, description='End Date', step=1)

Dropdown(description='Select time you prefer', options=('Please select', 'last 2 weeks', 'last 4 weeks', 'last…

changed 6 months


In [18]:
%run ./DataProcessing/CL_TimeFrame.ipynb

In [19]:
from datetime import date
from datetime import datetime

# get variables
organisation = selected_org.label
# machine_id = selected_machine.value
analysis_start_time = datetime.combine(DT_start.value, datetime.min.time()).timestamp()
analysis_end_time = datetime.combine(DT_end.value, datetime.min.time()).timestamp()

In [20]:
#  push to database

for machine_id in selected_machine.value:
    try:
        mycursor = mydb.cursor()

        sql = "INSERT INTO telematics_tasks (organisation, machine_id, analysis_start_time, analysis_end_time) VALUES (%s, %s, %s, %s)"
        val = [
          (organisation, machine_id, int(analysis_start_time),int(analysis_end_time)),]

        mycursor.executemany(sql, val)

        mydb.commit()

        print(mycursor.rowcount, "was inserted.")
    except:
        print('error in inserting to database')

In [22]:
def retrieve_tasks(mycursor) -> list:
    
    # execute your query
    mycursor.execute("SELECT * FROM telematics_tasks")

    # fetch all the matching rows
    result = mycursor.fetchall()

    listed_tasks = []

    # loop through the rows
    for row in result:
        listed_tasks.append(row)
    return listed_tasks

In [24]:
def generate_tasks_dialog(listed_tasks) -> None:
    
    w = widgets.Dropdown(
        options=['please select a task'] + listed_tasks,
        value='please select a task',
        description='Task:',
    )

    def on_change(change):
        if change['type'] == 'change' and change['name'] == 'value':
            print("changed to %s" % change['new'])

    w.observe(on_change)
    
    return w

# print(type(generate_tasks_dialog()))
display(generate_tasks_dialog(retrieve_tasks(mycursor)))

ProgrammingError: Cursor is not connected